# K-Nearest-Neighbors Membership Inference Experiment

In [1]:
import pandas as pd
import sklearn.ensemble as es
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
import logging

In [2]:
threads = 15

logging.basicConfig()

logger = logging.getLogger('xai-privacy')

In [3]:
from experiment_setup import run_all_experiments
from experiment_setup import get_heart_disease_dataset
from experiment_setup import get_census_dataset

In [4]:
DATASET_HALF = True

data_heart_dict, data_heart_num_dict, data_heart_cat_dict = get_heart_disease_dataset(halve_dataset=DATASET_HALF)
data_census_dict, data_census_num_dict, data_census_cat_dict = get_census_dataset(halve_dataset=DATASET_HALF)

data_heart = data_heart_dict['dataset']
outcome_name_heart = data_heart_dict['outcome']
numeric_features_heart = data_heart_dict['num']

census_num = data_census_num_dict['dataset']
outcome_name_census = data_census_num_dict['outcome']
numeric_features_census = data_census_num_dict['num']

Feature Age: removed 0 rows for missing values.
Feature RestingBP: removed 59 rows for missing values.
Feature Cholesterol: removed 27 rows for missing values.
Feature FastingBS: add unknown category 2.0
Feature RestingECG: add unknown category 3.0
Feature MaxHR: removed 0 rows for missing values.
Feature Oldpeak: removed 7 rows for missing values.
Feature ST_Slope: add unknown category 4.0
Feature CA: add unknown category 4.0
Feature Thal: add unknown category 8.0
Dropped 71 of 548
Dropped 72 of 548
Dropped 71 of 548
Dropped: 2399 of 32561
census: Dropped 1256 of 15081
num: Dropped 8827 of 15081
cat: Dropped 4850 of 15081


This notebook will go through the experiment for membership inference with KNN as an explanation. Membership inference means an attacker with access to the explanation can tell for a sample whether it was included in the training data or not.

The idea for KNN membership inference is as follows: Enter the given sample and check whether the sample is returned as one of its own nearest neighbors. If it is, it is part of the training data. Otherwise it is not.

First, we have to create our own wrapper class for the default scikit-learn KNN classifier. This is necessary because the scikit-learn implementation only returns the indices of the nearest neighbors. However, in order to be useful explanation to a user, the actual feature values of the nearest neighbors need to be returned. This is done by the wrapper class.

In [5]:
# Attack code must be imported so that multiprocessing pool works. Check out knn_attack.py for the implementation of the attack.
from knn_attack import KnnExplainer

Then, we implement the `train_explainer` and `membership_inference_attack_no_model_access` functions:

In [6]:
# Attack code must be imported so that multiprocessing pool works. Check out ice_attack.py for the implementation of the attack.
from knn_attack import KnnMembershipInference

# Dataset 1: Heart Disease

We now generate a KNN explaination for a random sample from the training data as a demonstration of the concept.

In [7]:
features = data_heart.drop(outcome_name_heart, axis=1)
labels = data_heart[outcome_name_heart]

# Train a random forest on training data.
model = es.RandomForestClassifier(random_state=0)
model = model.fit(features, labels)

# Train explainer
exp = KnnExplainer(data_heart, outcome_name_heart)

given_sample = features.sample()
print(f'Given sample: {given_sample.to_numpy()}')

pred = model.predict(given_sample)
print(f'Model prediction: {pred[0]}')

neighbors = exp.explain(given_sample)
print(f'Nearest Neighbors: \n {neighbors.to_numpy()}')

Given sample: [[ 59.   0.   4. 174. 249.   0.   0. 143.   1.   0.   2.   0.   3.]]
Model prediction: 1.0
Nearest Neighbors: 
 [[5.90e+01 0.00e+00 4.00e+00 1.74e+02 2.49e+02 0.00e+00 0.00e+00 1.43e+02
  1.00e+00 0.00e+00 2.00e+00 0.00e+00 3.00e+00 1.00e+00]
 [7.00e+01 1.00e+00 2.00e+00 1.56e+02 2.45e+02 0.00e+00 2.00e+00 1.43e+02
  0.00e+00 0.00e+00 1.00e+00 0.00e+00 3.00e+00 0.00e+00]
 [5.90e+01 1.00e+00 1.00e+00 1.78e+02 2.70e+02 0.00e+00 2.00e+00 1.45e+02
  0.00e+00 4.20e+00 3.00e+00 0.00e+00 7.00e+00 0.00e+00]
 [6.90e+01 1.00e+00 1.00e+00 1.60e+02 2.34e+02 1.00e+00 2.00e+00 1.31e+02
  0.00e+00 1.00e-01 2.00e+00 1.00e+00 3.00e+00 0.00e+00]
 [6.60e+01 1.00e+00 4.00e+00 1.60e+02 2.28e+02 0.00e+00 2.00e+00 1.38e+02
  0.00e+00 2.30e+00 1.00e+00 0.00e+00 6.00e+00 0.00e+00]]


We will now do a small proof of concept of the experiment with logging enabled to demonstrate how it works.

In [8]:
logger.setLevel(logging.DEBUG)

EXP = KnnMembershipInference(census_num, numeric_features_census, outcome_name_census, random_state=0)
EXP.membership_inference_experiment(num_queries=10, model=DecisionTreeClassifier(random_state=0), model_access=False, threads=1)

logger.setLevel(logging.INFO)

DEBUG:xai-privacy:Numeric Features: ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
DEBUG:xai-privacy:Categorical Features: []
DEBUG:xai-privacy:Removed 0 test samples due to unknown category.
DEBUG:xai-privacy:[[67  9  0  0 60  0]] taken from training data
DEBUG:xai-privacy:[[35 10  0  0 38  0]] taken from test data
DEBUG:xai-privacy:[[  35    9 7298    0   35    1]] taken from training data
DEBUG:xai-privacy:[[37  8  0  0 40  0]] taken from test data
DEBUG:xai-privacy:[[34 11  0  0 60  1]] taken from training data
DEBUG:xai-privacy:[[54 10  0  0 50  0]] taken from test data
DEBUG:xai-privacy:[[   60     9 15024     0    15     1]] taken from training data
DEBUG:xai-privacy:[[22 10  0  0 60  0]] taken from test data
DEBUG:xai-privacy:[[40  9  0  0 35  0]] taken from training data
DEBUG:xai-privacy:[[21  8  0  0 25  0]] taken from test data
DEBUG:xai-privacy:Checking sample 0: [67.  9.  0.  0. 60.  0.]
DEBUG:xai-privacy:Input sample:     age  education_num  c

Total time: 0.30s (training model: 0.03s, training explainer: 0.02s, experiment: 0.25s)
Accuracy: 1.0, precision: 1.0, recall: 1.0


The proof of concept should show that the membership inference function predicts membership very accurately.

Now we begin executing the actual experiment. We begin by defining the table that will hold the results for all our different experiment variations. Then we execute all variations of the experiment for this dataset. We vary the model between a decision tree, a random forest and a neural network. Each model uses the default configuration of scikit-learn.

In [9]:
dataset_dicts = [data_heart_dict, data_heart_num_dict, data_heart_cat_dict, data_census_dict, data_census_num_dict, data_census_cat_dict]

dt_dict = {'name': 'decision tree', 'model': DecisionTreeClassifier}
rf_dict = {'name': 'random forest', 'model': es.RandomForestClassifier}
nn_dict = {'name': 'neural network', 'model': MLPClassifier}

model_dicts = [dt_dict, rf_dict, nn_dict]

In [10]:
# This will run the experiment for each dataset and model combination

results = run_all_experiments(KnnMembershipInference, dataset_dicts, model_dicts, random_state=0, num_queries=None, model_access=False, threads=threads, repeat=10)

dataset: heart, model: decision tree (repetition 0)
Total time: 6.42s (training model: 0.03s, training explainer: 0.02s, experiment: 6.37s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 1)
Total time: 6.25s (training model: 0.02s, training explainer: 0.00s, experiment: 6.24s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 2)
Total time: 5.99s (training model: 0.02s, training explainer: 0.00s, experiment: 5.98s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 3)
Total time: 5.94s (training model: 0.02s, training explainer: 0.00s, experiment: 5.93s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 4)
Total time: 6.64s (training model: 0.02s, training explainer: 0.00s, experiment: 6.62s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 5)
Total time: 5.66s (training model

c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 4.52s (training model: 0.65s, training explainer: 0.00s, experiment: 3.88s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: neural network (repetition 1)


c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 4.72s (training model: 0.70s, training explainer: 0.00s, experiment: 4.01s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: neural network (repetition 2)


...

Total time: 32.67s (training model: 11.71s, training explainer: 0.00s, experiment: 20.96s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census categorical, model: neural network (repetition 8)


c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 32.47s (training model: 11.55s, training explainer: 0.01s, experiment: 20.91s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census categorical, model: neural network (repetition 9)


c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 33.07s (training model: 11.81s, training explainer: 0.01s, experiment: 21.25s)
Accuracy: 1.0, precision: 1.0, recall: 1.0


# Results

The results of all variations of the membership inference experiment with KNN. In every experiment, we executed the membership inference attack on each sample of the training data and each sample of the test data. Both datasets are of equal size and originate from the same source dataset.

Accuracy is the percentage of samples whose membership (true or false) was correctly inferred. An algorithm guessing at random would achieve an accuracy of 50 percent.

Precision is the percentage of predicted training samples that is actually in the training data.

Recall is the percentage of training samples whose membership (true) was correctly inferred.

In [11]:
results

,dataset,model,repetition,accuracy,precision,recall
0,heart,decision tree,0,1.0,1.0,1.0
1,heart,decision tree,1,1.0,1.0,1.0
2,heart,decision tree,2,1.0,1.0,1.0
3,heart,decision tree,3,1.0,1.0,1.0
4,heart,decision tree,4,1.0,1.0,1.0
...,...,...,...,...,...,...
175,census categorical,neural network,5,1.0,1.0,1.0
176,census categorical,neural network,6,1.0,1.0,1.0
177,census categorical,neural network,7,1.0,1.0,1.0
178,census categorical,neural network,8,1.0,1.0,1.0


In [12]:
file_name = 'results/1-4-knn-membership-inference-results'
if DATASET_HALF:
    file_name += '_dataset_size_halved'
results.to_csv(file_name + '.csv', index=False, na_rep='NaN', float_format='%.3f')

# Discussion

Just as expected, the attack has an accuracy of 100%. No false positives or false negatives occur.